def userdata( User_id : str ): 
Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [1]:
#importar las librerias
import pandas as pd
import numpy as np

In [11]:
# Leer dataframe de videjuegos solo con las columnas que se necesitan
columns = ['id', 'price']
df_precios = pd.read_csv("steam_games.csv", usecols=columns)
df_precios.head()

,price,id
0,4.99,761140.0
1,Free To Play,643980.0
2,Free to Play,670290.0
3,0.99,767400.0
4,2.99,773570.0


In [12]:
# Verificar que los datatypes estén bien y no haya nulos
df_precios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   30758 non-null  object 
 1   id      32133 non-null  float64
dtypes: float64(1), object(1)
memory usage: 502.2+ KB


In [13]:
# Función para corregir el precio, en el que aparecen valores numéricos y en formato string
def fix_price(df):

    errors_list = []
    for i in df['price']:
        try:
            float(i)
        except:
            errors_list.append(i)

    errors = set(errors_list)
    #uniques_not_free = ['Starting at $499.00', 'Starting at $449.00']
    df['price_fixed'] = df['price'].apply(lambda x: 0 if x in errors 
                                                        else 499.0 if x=='Starting at $499.00'
                                                        else 449.0 if x=='Starting at $449.00'
                                                        else x)
    df['price_fixed'] = df['price_fixed'].astype(float)
    return df

In [14]:
# Aplicar la función al dataframe
df_precios = fix_price(df_precios)
# Renombrar la columna de id 
df_precios.rename(columns={'id':'item_id'},inplace=True)
# Eliminar la antigua columna de precio
df_precios.drop(columns='price', inplace=True)
df_precios.head()

,item_id,price_fixed
0,761140.0,4.99
1,643980.0,0.00
2,670290.0,0.00
3,767400.0,0.99
4,773570.0,2.99


In [15]:
# Verificar nulos
df_precios.isna().sum()

item_id           2
price_fixed    1377
dtype: int64

In [16]:
# Reemplazar nulos en el precio con 0
df_precios.fillna(0, inplace=True)

In [55]:
# Leer el daraframe de user items solo con las columnas que se necesitan
columns = ['steam_id', 'item_id', 'items_count']
user_items = pd.read_csv("game_items.csv", usecols=columns)
user_items.head()
user_items.rename(columns={"steam_id":"user_id"},inplace=True)
user_items["user_id"]=user_items["user_id"].astype(str)

In [36]:
# Hacer un merge (left join) de ambos dataframes on item_id
df_merged = user_items.merge(df_precios, on='item_id', how='left')
df_merged.sample(10)

,user_id,items_count,item_id,price_fixed
5027403,76561198091773972,26,219540,NaN
996483,76561197993302844,119,18500,9.99
2033537,76561197987617385,171,57690,14.99
4413287,76561198049036232,66,8190,14.99
1860963,76561198215611220,102,291480,0.00
2570157,76561197992708849,143,8930,29.99
3662848,76561198005580769,117,40100,12.99
4071280,76561198019022742,898,328990,2.99
5113390,76561198118198379,76,21090,54.99
3966200,76561197999596498,60,32430,19.99


In [49]:
# Group by usuario para obtener el total de dinero gastado en videojuegos
user_spent = df_merged.groupby('user_id')['price_fixed'].agg('sum')
user_spent = pd.DataFrame(user_spent)
user_spent.reset_index(inplace=True)
user_spent["user_id"]=user_spent["user_id"].astype(str)

In [50]:
user_spent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70912 entries, 0 to 70911
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_id      70912 non-null  object 
 1   price_fixed  70912 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.1+ MB


In [39]:
# Leer dataframe de reviews solo con las columnas que se necesitan
columns = ['user_id', 'recommend']
user_revs = pd.read_csv('game_reviews.csv', usecols=columns)
user_revs.sample(5)

,user_id,recommend
24729,nytewolfaa2k7,False
41480,76561198050680379,True
54863,76561198082945647,True
8404,aket87,True
34802,76561198081265170,True


In [40]:
# Calcular el porcentaje de recomendación para cada usuario
# Crear un df vacío
user_gb_count_revs = pd.DataFrame()
# Agrupar por user_id y agregar por conteo de recommend para obtener el total de reviews
user_gb_count_revs['count_total'] = user_revs.groupby('user_id')['recommend'].agg('count')
# Contar solo los recommend = True por usuario
user_gb_count_revs['count_true'] = user_revs[user_revs['recommend'] == True].groupby('user_id')['recommend'].agg(count_revs='count')
user_gb_count_revs.reset_index(inplace=True)
# Calcular la proporción de True entre el total
user_gb_count_revs['reco_porcentaje'] = user_gb_count_revs['count_true'] / user_gb_count_revs['count_total'] * 100
user_gb_count_revs.fillna(0,inplace=True)
user_gb_count_revs.sample(5)

,user_id,count_total,count_true,reco_porcentaje
10996,76561198104824869,3,3.0,100.0
6365,76561198068939671,1,1.0,100.0
7848,76561198077596124,10,5.0,50.0
24968,viggie12,1,1.0,100.0
20751,heetesh,3,3.0,100.0


In [42]:
user_gb_count_revs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25485 entries, 0 to 25484
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          25485 non-null  object 
 1   count_total      25485 non-null  int64  
 2   count_true       25485 non-null  float64
 3   reco_porcentaje  25485 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 796.5+ KB


In [51]:
# Hacer un merge (left join) de los df de reviews y user_spent anteriormente creados
df_merge2 = user_gb_count_revs.merge(user_spent, on='user_id',how='left' )
df_merge2.rename(columns={'price_fixed':'total_spent'},inplace=True)
df_merge2.sample(10)

,user_id,count_total,count_true,reco_porcentaje,total_spent
15173,LucidKing,3,2.0,66.666667,NaN
11400,76561198124366634,1,1.0,100.000000,346.72
9412,76561198087992251,1,1.0,100.000000,534.50
9653,76561198089820105,1,1.0,100.000000,1303.73
14683,Jupacka,1,1.0,100.000000,NaN
13878,GAycuntt,1,1.0,100.000000,NaN
17357,TheJaguar8,1,1.0,100.000000,NaN
22234,maxyrogers,1,1.0,100.000000,NaN
3996,76561198053439793,1,1.0,100.000000,NaN
9167,76561198086028360,1,1.0,100.000000,74.95


In [57]:
# # Se realiza otro merge con la tabla de user_items para obtener el total de items por usuario
df_merge3 = df_merge2.merge(user_items[['user_id','items_count']].drop_duplicates(),on='user_id',how='left')
# Se eliminan columnas innecesarias
df_merge3 = df_merge3.drop(['count_total','count_true'],axis=1)
# Renombrar columnas
df_merge3.rename(columns={
    'reco_porcentaje':'porcentaje_recomendacion', 'total_spent':'cantidad_gastada', 'items_count':'cantidad_items'},inplace=True)

In [59]:
# Guardar el dataframe final como csv para consumir en la API
df_merge3.to_csv('df-funcion-1-1.csv',index=False)

In [ ]:
# Función de la consulta para obtener el dinero total gastado, cantidad de items 
# y porcentaje de recomendación por usuario
def userdata(user_id : str):
    df_user = pd.read_csv('df-funcion-1-1.csv')
    user_data = df_user[df_user['user_id'] == user_id]
    
    return user_data.to_json(orient='records')

In [64]:
#usar solo lo que se necesita del game items
df1 = pd.read_csv("game_items.csv",usecols=["steam_id","items_count"])

In [65]:
#elimianr los duplicados
df1=df1.drop_duplicates(keep='first')
#convertir la columna steamid a string
df1["steam_id"]=df1["steam_id"].astype(str)

In [66]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70912 entries, 0 to 5153202
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   steam_id     70912 non-null  object
 1   items_count  70912 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.6+ MB


In [67]:
#volverlo un archivo
df1.to_csv("funcion1-2.csv")

In [68]:
#funcion pára hacer una lista de strings
list_string=[]
for i in df_precios["price"]:
    try:
        float(i)
    except:
        list_string.append(i)
       
list_string=set(list_string)


In [69]:
#convertir los string en lista
list_string=list(list_string)

In [70]:
list_string

['Free',
 'Starting at $499.00',
 'Install Theme',
 'Starting at $449.00',
 'Free to Play',
 'Third-party',
 'Free To Play',
 'Free to Try',
 'Install Now',
 'Free to Use',
 'Play WARMACHINE: Tactics Demo',
 'Free HITMAN™ Holiday Pack',
 'Free Demo',
 'Free Movie',
 'Play Now',
 'Free Mod',
 'Play for Free!',
 'Play the Demo']

In [71]:
#asignar los valores a los precios que inicialmente son string
df_precios['price'] = df_precios['price'].apply(lambda x: 0 if x in list_string
                                                        else 499.0 if x=='Starting at $499.00'
                                                        else 449.0 if x=='Starting at $449.00'
                                                        else x)


In [72]:
df_precios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   30758 non-null  object 
 1   id      32133 non-null  float64
dtypes: float64(1), object(1)
memory usage: 502.2+ KB


In [73]:
#convertir los precios a float
df_precios["price"]=df_precios["price"].astype(float)

In [74]:
#llenar los nan
df_precios["price"].fillna(df_precios['price'].median(), inplace=True)
df_precios["id"].fillna(method="ffill",inplace=True)

In [75]:
#ver si hay nulos aun
df_precios.isna().sum()

price    0
id       0
dtype: int64

In [76]:
#cambiar el nombre de la columna
df_precios.rename(columns={"id":"item_id"}, inplace=True)

#cambiar el tipo de la columna
df_precios["item_id"]=df_precios["item_id"].astype(str)

In [78]:
#cambiar el tipo de la columna
df_recomendados["item_id"]=df_recomendados["item_id"].astype(str)

In [105]:
#unir las tablas recomendados y precios
df_merged=df_recomendados.merge(df_precios,on='item_id', how='left')

In [106]:
#reemplzar los nulos con la media
df_merged["price"].fillna(df_merged['price'].median(), inplace=True)


In [107]:
#construir las agrupaciones para las recomendaciones
cuenta_total=df_merged.groupby("user_id")["recommend"].count()
cuenta_true=df_merged[df_merged["recommend"]==True].groupby("user_id")["recommend"].count()
#convertirlo en dataframes
cuenta_total=pd.DataFrame(cuenta_total)
cuenta_true=pd.DataFrame(cuenta_true)
#renombrar las columnas para que tengan otros conmbres las columnas
cuenta_total.rename(columns={"recommend":"total_recommed"},inplace=True)
cuenta_true.rename(columns={"recommend":"total_True"},inplace=True)
#añadir un indice nuevo
cuenta_total=cuenta_total.reset_index()
cuenta_true=cuenta_true.reset_index()


In [108]:
#crear tabla con los datos que se necesita
df5=cuenta_total.merge(cuenta_true, on="user_id", how='left')
#sacar la columna que necesito
df5["porcentaje"]=round((df5["total_True"]/df5["total_recommed"])*100,2)

In [109]:
df5.drop(columns=["total_recommed","total_True"],inplace=True)

In [143]:
df5.to_csv("funcion1-3.csv")

In [111]:
df_merged.drop(columns=["item_id","recommend"], inplace=True)

In [117]:
df_merged.to_csv("funcion1-3.csv")

In [115]:
#enviar el csv
df_merged.to_csv("funcion1-1.csv")

In [149]:
#gasto por id
gastoxid=df_merged[df_merged["user_id"]=="76561198075565282"]["price"].agg(sum)

In [150]:
gastoxid

79.97

In [138]:
#porcentaje de recomendacion
recommend= df5[df5["user_id"]=="76561197970982479"]["porcentaje"].values[0]

In [139]:
recommend

100.0

In [28]:
#cantidad de items por usuario 
itemsxusuario=df1[df1["steam_id"]=="76561197970982479"]["items_count"][0]

In [29]:
itemsxusuario

277

In [146]:
df_merged

,user_id,price
0,76561197970982479,19.99
1,76561197970982479,9.99
2,76561197970982479,9.99
3,js41637,9.99
4,js41637,19.99
...,...,...
59330,76561198312638244,9.99
59331,76561198312638244,19.99
59332,LydiaMorley,0.00
59333,LydiaMorley,14.99


In [151]:
def userdata( User_id : str ): 
    
    df2=pd.read_csv("funcion1-1.csv")
    df3=pd.read_csv("funcion1-2.csv")
    df4=pd.read_csv("funcion1-3.csv")
    #gasto por id
    gastoxid=df2[df2["user_id"]==User_id]["price"].agg(sum)
    #%de recomendaciones
    recommend= df4[df4["user_id"]=="76561197970982479"]["porcentaje"].values[0]
   
    #cantidad de items por usuario 
    itemsxusuario= df3[df3["steam_id"]==User_id]["items_count"][0]
 
    resultado={"el gasto por el usuario":gastoxid, "el procentaje de recomendacion es" :recommend, "el numero de items por usuario ":itemsxusuario}

    return resultado

In [152]:
userdata("76561197970982479")

KeyError: 0